In [1]:
import re

from itertools import cycle, combinations, permutations, tee
from collections import Counter, defaultdict, deque
from io import StringIO

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def read_input(day, fn=str.strip):
    """//
    Return a list of the input lines mapped by fn
    
    example: 
    >>> read_input('01', int)  # read input file, map all lines to int
    
    Inspired by Peter Norvig: https://github.com/norvig/pytudes
    
    """
    return list(map(fn, open(f'input\{day}.txt')))

def all_integers(s):
    """return all integers from a string"""
    return tuple(map(int, re.findall(r'-?\d+', s)))

# Day 20

In [7]:
testcase = open('input\\20-test.txt').read()
testcase[:30]

'Tile 2311:\n..##.#..#.\n##..#...'

In [23]:
testtile = """..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###"""

testtile = testtile.split('\n')

In [51]:
def get_edges(tile):
    """return the edges (and reversed egdes) of tile"""
    left_side = ''.join(l[0] for l in tile)
    right_side = ''.join(l[-1] for l in tile)
                        
    return (tile[0], tile[0][::-1], tile[-1],tile[-1][::-1],
            right_side, right_side[::-1], left_side, left_side[::-1])

get_edges(testtile)

('..##.#..#.',
 '.#..#.##..',
 '..###..###',
 '###..###..',
 '...#.##..#',
 '#..##.#...',
 '.#####..#.',
 '.#..#####.')

In [78]:
def parse_tiles(l):
    tiles = {}
    edges = defaultdict(set)
    
    for tile in l.split('\n\n'):
        header, *tile = tile.split('\n')
        assert header[:4] == 'Tile'
        tile_id = int(re.findall(r'-?\d+', header)[0])
        tiles[tile_id] = tile
        for edge in get_edges(tile):
            edges[edge].add(tile_id)
        
    return tiles, edges

tiles, edge_dict = parse_tiles(testcase)
#parse_tiles(testcase)

In [79]:
def find_neighbours(tiles, edge_dict):
    tree = defaultdict(set)
    for tile_id, tile in tiles.items():
        for edge in get_edges(tile):
            for other_tile_id in edge_dict[edge]:
                if other_tile_id == tile_id:
                    continue
                else: 
                    tree[tile_id].add(other_tile_id)
    return tree

find_neighbours(tiles, edge_dict)

defaultdict(set,
            {2311: {1427, 1951, 3079},
             1951: {2311, 2729},
             1171: {1489, 2473},
             1427: {1489, 2311, 2473, 2729},
             1489: {1171, 1427, 2971},
             2473: {1171, 1427, 3079},
             2971: {1489, 2729},
             2729: {1427, 1951, 2971},
             3079: {2311, 2473}})

In [80]:
def partA(l):
    tiles, edge_dict = parse_tiles(l)
    edges = find_neighbours(tiles, edge_dict)
    corners = [t for t, edges in edges.items() if len(edges) == 2]
    res = 1
    for tile_id in corners:
        res *= tile_id
    return res

partA(testcase)

20899048083289

In [81]:
inp = open('input\\20.txt').read()
partA(inp)

66020135789767

# part B